In [ ]:
import numpy as np
import cvxpy as cvx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
import dpilqr as dec

In [ ]:
"""Run SCP in receding horizon"""
n_agents = 2
n_states = 12
count = 0
si = s0
obj_list = []

N = 15
# u_try = np.array([0, 0, 0, 4.9])

u_try = np.array([0, 0, 0, 4.9,
                  0, 0, 0, 4.9])

u_bar = np.tile(u_try ,(N,1))
s_bar = np.zeros((N + 1, n_agents*n_states))
s_bar[0] = s0
for k in range(N):
    s_bar[k+1] = fd(s_bar[k], u_bar[k])

X_trj =  np.zeros((0, n_agents*n_states))  #Full trajectory over entire problem horizon (not just a single prediction horizon)
STEP_SIZE=1

iterate = 0
s_prev = None
obj_prev = np.inf
tol = 5e-1
x_dims = [n_states]*n_agents

while not np.all(dec.distance_to_goal(si, s_goal.reshape(1,-1), n_agents ,n_states,n_d= 3) < 0.1) :
    s, u, obj = scp_iteration(fd, P, Q, R, N, s_bar, u_bar, s_goal, si,  ρ, iterate, s_prev, n_agents)

    s_prev = s
    
    diff_obj = np.abs(obj - obj_prev)
    print(f'current diff_obj is {diff_obj}')
    if diff_obj < tol:
            
        print('SCP converged')
        break
    
    else:
        obj_prev = obj
        #Re-initialize nominal trajectory to shift prediction horizon
        s_bar = np.zeros((N + 1, n))
        s_bar[0] = s[STEP_SIZE]
        u_bar = np.tile(u_try ,(N,1))
        u_bar[0] = u[STEP_SIZE-1]

        count +=1
        
        obj_list.append(obj)
        
        print(f'current objective value is {obj}!\n')

        X_trj = np.r_[X_trj, s[:STEP_SIZE]]
        print(f'X_trj has shape {X_trj.shape}\n')

        if count >=60:
            print('max iteration reached')
            break

# Plot state and control trajectories
print(f'Full trajectory has shape {X_trj.shape}')
fig = plt.figure(dpi=200)
dec.plot_solve(X_trj,0,s_goal,x_dims,n_d = 3)
plt.savefig('1quad_rhc_SCP.png')
